In [42]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

# 设备配置
compute_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"运行在: {compute_device}")

# 路径配置
training_images_dir = '/kaggle/input/4-task/detection/train'
testing_images_dir = '/kaggle/input/4-task/detection/test'
training_labels_file = '/kaggle/input/4-task/detection/fovea_localization_train_GT.csv'
submission_template = '/kaggle/input/4-task/detection/sample_submission.csv'

# 参数配置
IMAGE_DIMENSION = 256
BATCH_NUM = 8
TRAINING_EPOCHS = 50
LR_VALUE = 1e-4
HEATMAP_SIGMA = 5


# -------------------------- 1. 网络架构定义 --------------------------
class ConvBlock(nn.Module):
    """双卷积模块"""

    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_block(x)


class EncodingPath(nn.Module):
    """编码路径模块"""

    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.encoding_step = nn.Sequential(
            nn.MaxPool2d(2),
            ConvBlock(input_channels, output_channels)
        )

    def forward(self, x):
        return self.encoding_step(x)


class DecodingPath(nn.Module):
    """解码路径模块"""

    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.upsample = nn.ConvTranspose2d(input_channels, input_channels // 2, kernel_size=2, stride=2)
        self.conv_block = ConvBlock(input_channels, output_channels)

    def forward(self, x_up, x_skip):
        x_up = self.upsample(x_up)
        # 处理尺寸差异
        diff_h = x_skip.size()[2] - x_up.size()[2]
        diff_w = x_skip.size()[3] - x_up.size()[3]
        x_up = nn.functional.pad(x_up, [diff_w // 2, diff_w - diff_w // 2,
                                        diff_h // 2, diff_h - diff_h // 2])
        merged = torch.cat([x_skip, x_up], dim=1)
        return self.conv_block(merged)


class FinalConv(nn.Module):
    """最终卷积层"""

    def __init__(self, input_channels, output_channels):
        super(FinalConv, self).__init__()
        self.conv_layer = nn.Conv2d(input_channels, output_channels, kernel_size=1)

    def forward(self, x):
        return self.conv_layer(x)


class SegmentationNetwork(nn.Module):
    """分割网络主结构"""

    def __init__(self, num_channels=1, num_classes=1):
        super(SegmentationNetwork, self).__init__()
        self.num_channels = num_channels
        self.num_classes = num_classes

        self.initial_conv = ConvBlock(num_channels, 64)
        self.encode1 = EncodingPath(64, 128)
        self.encode2 = EncodingPath(128, 256)
        self.encode3 = EncodingPath(256, 512)
        self.encode4 = EncodingPath(512, 1024)
        self.decode1 = DecodingPath(1024, 512)
        self.decode2 = DecodingPath(512, 256)
        self.decode3 = DecodingPath(256, 128)
        self.decode4 = DecodingPath(128, 64)
        self.final_conv = FinalConv(64, num_classes)

    def forward(self, x):
        conv1 = self.initial_conv(x)
        conv2 = self.encode1(conv1)
        conv3 = self.encode2(conv2)
        conv4 = self.encode3(conv3)
        conv5 = self.encode4(conv4)

        up1 = self.decode1(conv5, conv4)
        up2 = self.decode2(up1, conv3)
        up3 = self.decode3(up2, conv2)
        up4 = self.decode4(up3, conv1)

        output = self.final_conv(up4)
        return output


# -------------------------- 2. 数据预处理工具 --------------------------
def generate_heatmap(dimensions, center_point, sigma=HEATMAP_SIGMA):
    """
    生成高斯热力图
    :param dimensions: 热力图尺寸 (高度, 宽度)
    :param center_point: 中心坐标 (x, y)
    :param sigma: 高斯标准差
    :return: 归一化热力图
    """
    height, width = dimensions
    center_x, center_y = center_point

    # 生成坐标网格
    xx, yy = np.meshgrid(np.arange(width), np.arange(height))

    # 计算高斯分布
    heatmap_data = np.exp(-((xx - center_x) ** 2 + (yy - center_y) ** 2) / (2 * sigma ** 2))

    # 归一化处理
    heatmap_data = heatmap_data / np.max(heatmap_data)

    return heatmap_data


class MedicalImageDataset(Dataset):
    """医学图像数据集"""

    def __init__(self, image_paths, target_points=None, transform_ops=None, testing_mode=False):
        self.image_paths = image_paths
        self.target_points = target_points
        self.transform_ops = transform_ops
        self.testing_mode = testing_mode

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # 读取图像
        img_path = self.image_paths[index]
        image_data = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # 调整尺寸
        image_data = cv2.resize(image_data, (IMAGE_DIMENSION, IMAGE_DIMENSION))

        # 应用变换
        if self.transform_ops:
            image_data = self.transform_ops(image_data)
        else:
            # 保证返回 torch.Tensor
            image_data = torch.from_numpy(image_data).unsqueeze(0).float() / 255.0

        if self.testing_mode:
            # 测试模式只返回图像和ID
            image_id = os.path.basename(img_path).split('.')[0]
            return image_data, image_id

        # 训练模式返回图像和热力图
        target_x, target_y = self.target_points[index]

        # 坐标缩放
        # 读取原始尺寸（单独读取以保持原始比例）
        orig_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        orig_h, orig_w = orig_image.shape

        # 缩放到训练使用的热力图/输入尺寸
        scaled_x = target_x * (IMAGE_DIMENSION / orig_w)
        scaled_y = target_y * (IMAGE_DIMENSION / orig_h)

        # 创建热力图
        heatmap = generate_heatmap((IMAGE_DIMENSION, IMAGE_DIMENSION), (scaled_x, scaled_y))
        heatmap = torch.from_numpy(heatmap).float().unsqueeze(0)

        # 返回：图像张量、热力图、原始尺寸、缩放后的标签（用于像素误差计算）
        sizes_tensor = torch.tensor([orig_w, orig_h], dtype=torch.float32)
        scaled_tensor = torch.tensor([scaled_x, scaled_y], dtype=torch.float32)
        return image_data, heatmap, sizes_tensor, scaled_tensor


# -------------------------- 3. 模型训练和预测 --------------------------
def setup_training_data():
    """准备训练和验证数据"""
    # 读取标签文件
    label_data = pd.read_csv(training_labels_file)

    image_path_list = []
    coordinate_list = []

    for _, record in label_data.iterrows():
        img_id_val = int(record['data'])
        fovea_x_val = record['Fovea_X']
        fovea_y_val = record['Fovea_Y']

        image_name = f"{img_id_val:04d}.jpg"
        full_image_path = os.path.join(training_images_dir, image_name)

        if os.path.exists(full_image_path):
            image_path_list.append(full_image_path)
            coordinate_list.append((fovea_x_val, fovea_y_val))

    # 数据集划分
    train_paths, valid_paths, train_coords, valid_coords = train_test_split(
        image_path_list, coordinate_list, test_size=0.2, random_state=42
    )

    # 数据变换
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    # 创建数据集
    training_dataset = MedicalImageDataset(train_paths, train_coords, data_transforms)
    validation_dataset = MedicalImageDataset(valid_paths, valid_coords, data_transforms)

    # 创建数据加载器
    training_loader = DataLoader(training_dataset, batch_size=BATCH_NUM, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=BATCH_NUM, shuffle=False)

    return training_loader, validation_loader


def execute_training():
    """执行模型训练"""
    # 准备数据
    train_data_loader, valid_data_loader = setup_training_data()

    # 初始化模型组件
    # constructor uses parameter names `num_channels` and `num_classes`
    network_model = SegmentationNetwork(num_channels=1, num_classes=1).to(compute_device)
    loss_function = nn.MSELoss()
    model_optimizer = optim.Adam(network_model.parameters(), lr=LR_VALUE)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(model_optimizer, 'min', patience=5, factor=0.5)

    # 训练过程
    best_validation_score = float('inf')
    best_pixel_rmse = float('inf')

    for epoch_counter in range(TRAINING_EPOCHS):
        # 训练阶段
        network_model.train()
        cumulative_train_loss = 0.0
        train_batch_scores = []
        train_pixel_errors = []

        progress_bar = tqdm(train_data_loader, desc=f'训练轮次 {epoch_counter + 1}/{TRAINING_EPOCHS}')
        for batch_index, batch in enumerate(progress_bar):
            # 解包批次：期望 (imgs, heats, sizes_tensor, scaled_tensor)
            if len(batch) == 2:
                batch_images, batch_heatmaps = batch
                batch_sizes = torch.tensor([[IMAGE_DIMENSION, IMAGE_DIMENSION]] * batch_images.size(0), dtype=torch.float32)
                batch_scaled = torch.full((batch_images.size(0), 2), float('nan'), dtype=torch.float32)
            else:
                batch_images, batch_heatmaps, batch_sizes, batch_scaled = batch

            # 确保 sizes/scaled 为 tensor
            if not torch.is_tensor(batch_sizes):
                batch_sizes = torch.tensor(batch_sizes, dtype=torch.float32)
            if not torch.is_tensor(batch_scaled):
                batch_scaled = torch.tensor(batch_scaled, dtype=torch.float32)

            batch_images = batch_images.to(compute_device)
            batch_heatmaps = batch_heatmaps.to(compute_device)

            # 前向计算
            model_outputs = network_model(batch_images)
            batch_loss = loss_function(model_outputs, batch_heatmaps)
            batch_mse = batch_loss.item()
            # 计算RMSE（每批次）
            batch_rmse = float(np.sqrt(batch_mse))
            train_batch_scores.append(batch_mse)

            # 反向传播
            model_optimizer.zero_grad()
            batch_loss.backward()
            model_optimizer.step()

            cumulative_train_loss += batch_loss.item() * batch_images.size(0)
            # 计算像素级误差：将heatmap的最大值位置映射回原始像素并与真实像素比较
            try:
                # model_outputs 形状 [B, C, H, W] ；假设 C==1
                b, c, h, w = model_outputs.shape
                flat_idx = model_outputs.view(b, -1).argmax(dim=1)
                pred_y = (flat_idx // w).cpu().numpy()
                pred_x = (flat_idx % w).cpu().numpy()

                # 逐样本计算像素MSE
                batch_pixel_mses = []
                for i in range(b):
                    orig_w = float(batch_sizes[i, 0].item())
                    orig_h = float(batch_sizes[i, 1].item())

                    # 将预测坐标从 heatmap 尺度映射回原始像素尺度（heatmap 宽高为 w,h）
                    pred_px = float(pred_x[i]) * (orig_w / w)
                    pred_py = float(pred_y[i]) * (orig_h / h)

                    true_scaled_x = batch_scaled[i, 0].item()
                    true_scaled_y = batch_scaled[i, 1].item()

                    if np.isnan(true_scaled_x) or np.isnan(true_scaled_y):
                        # 缺失标签则跳过该样本
                        continue

                    # true_scaled is in IMAGE_DIMENSION grid, map to original pixels
                    true_px = float(true_scaled_x) * (orig_w / IMAGE_DIMENSION)
                    true_py = float(true_scaled_y) * (orig_h / IMAGE_DIMENSION)

                    pixel_mse = ((pred_px - true_px) ** 2 + (pred_py - true_py) ** 2) / 2.0
                    batch_pixel_mses.append(pixel_mse)

                batch_pixel_mse = float(np.mean(batch_pixel_mses))
                batch_pixel_rmse = float(np.sqrt(batch_pixel_mse))
                train_pixel_errors.extend(batch_pixel_mses)
            except Exception:
                batch_pixel_rmse = float('nan')

            progress_bar.set_postfix({'批次MSE': f'{batch_mse:.6f}',
                                      '批次RMSE': f'{batch_rmse:.6f}',
                                      '批次像素RMSE': f'{batch_pixel_rmse:.2f}',
                                      '平均损失': f'{cumulative_train_loss / ((batch_index + 1) * batch_images.size(0)):.6f}'})

        # 计算训练集平均MSE
        avg_train_mse = cumulative_train_loss / len(train_data_loader.dataset)
        avg_train_rmse = float(np.sqrt(avg_train_mse))
        avg_train_pixel_rmse = float(np.sqrt(np.mean(train_pixel_errors))) if len(train_pixel_errors) > 0 else float('nan')

        # 验证阶段
        network_model.eval()
        cumulative_valid_loss = 0.0
        valid_batch_scores = []
        valid_pixel_errors = []

        with torch.no_grad():
            # Validation batching mirrors training batching: dataset returns (img, heatmap, (orig_w, orig_h), (scaled_x, scaled_y))
            for batch in valid_data_loader:
                # 解包批次（兼容旧签名）
                if len(batch) == 2:
                    imgs, heats = batch
                    sizes = [(IMAGE_DIMENSION, IMAGE_DIMENSION)] * imgs.size(0)
                    scaled = [None] * imgs.size(0)
                else:
                    imgs, heats, sizes, scaled = batch

                imgs = imgs.to(compute_device)
                heats = heats.to(compute_device)

                valid_outputs = network_model(imgs)
                valid_loss = loss_function(valid_outputs, heats)
                valid_batch_mse = valid_loss.item()
                valid_batch_rmse = float(np.sqrt(valid_batch_mse))
                valid_batch_scores.append(valid_batch_mse)

                # 计算像素级误差
                try:
                    b, c, h, w = valid_outputs.shape
                    flat_idx = valid_outputs.view(b, -1).argmax(dim=1)
                    pred_y = (flat_idx // w).cpu().numpy()
                    pred_x = (flat_idx % w).cpu().numpy()

                    for i in range(b):
                        orig_w = float(sizes[i, 0].item()) if torch.is_tensor(sizes) else float(sizes[i][0])
                        orig_h = float(sizes[i, 1].item()) if torch.is_tensor(sizes) else float(sizes[i][1])

                        pred_px = float(pred_x[i]) * (orig_w / w)
                        pred_py = float(pred_y[i]) * (orig_h / h)

                        if torch.is_tensor(scaled):
                            true_scaled_x = scaled[i, 0].item()
                            true_scaled_y = scaled[i, 1].item()
                        else:
                            true_scaled_x, true_scaled_y = scaled[i]

                        if np.isnan(true_scaled_x) or np.isnan(true_scaled_y):
                            continue

                        true_px = float(true_scaled_x) * (orig_w / IMAGE_DIMENSION)
                        true_py = float(true_scaled_y) * (orig_h / IMAGE_DIMENSION)

                        pixel_mse = ((pred_px - true_px) ** 2 + (pred_py - true_py) ** 2) / 2.0
                        valid_pixel_errors.append(pixel_mse)
                except Exception:
                    pass

                cumulative_valid_loss += valid_loss.item() * imgs.size(0)

        # 计算验证集平均MSE
        avg_valid_mse = cumulative_valid_loss / len(valid_data_loader.dataset)
        avg_valid_rmse = float(np.sqrt(avg_valid_mse))
        avg_valid_pixel_rmse = float(np.sqrt(np.mean(valid_pixel_errors))) if len(valid_pixel_errors) > 0 else float('nan')
        if len(valid_pixel_errors) == 0:
            print("警告：验证集像素误差计算为空，跳过保存")
            continue

        # 学习率调整
        lr_scheduler.step(avg_valid_mse)

        # 保存最佳模型
        if avg_valid_pixel_rmse < best_pixel_rmse and not np.isnan(avg_valid_pixel_rmse):
            best_pixel_rmse = avg_valid_pixel_rmse
            torch.save(network_model.state_dict(), 'optimal_model_weights.pth')
            print(f"✓ 保存最优模型，像素RMSE: {best_pixel_rmse:.2f}，验证MSE: {avg_valid_mse:.8f}")

        # 输出训练统计信息
        print(f'\n====== 第 {epoch_counter + 1} 轮统计 ======')
        print(f'训练集 - 平均MSE: {avg_train_mse:.8f}, 平均RMSE: {avg_train_rmse:.8f}, 平均像素RMSE: {avg_train_pixel_rmse:.2f}, 最小批次MSE: {min(train_batch_scores):.8f}, '
              f'最大批次MSE: {max(train_batch_scores):.8f}')
        print(f'验证集 - 平均MSE: {avg_valid_mse:.8f}, 平均RMSE: {avg_valid_rmse:.8f}, 平均像素RMSE: {avg_valid_pixel_rmse:.2f}, 最小批次MSE: {min(valid_batch_scores):.8f}, '
              f'最大批次MSE: {max(valid_batch_scores):.8f}')
        print(f'=================================\n')

    return network_model


def estimate_coordinates(trained_model, image_path):
    """
    预测图像中目标坐标
    :param trained_model: 训练好的模型
    :param image_path: 图像路径
    :return: 原始尺寸下的(x, y)坐标
    """
    # 读取并预处理图像
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    orig_height, orig_width = original_image.shape

    resized_image = cv2.resize(original_image, (IMAGE_DIMENSION, IMAGE_DIMENSION))
    inference_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image_tensor = inference_transform(resized_image).unsqueeze(0).to(compute_device)

    # 预测热力图
    trained_model.eval()
    with torch.no_grad():
        predicted_heatmap = trained_model(image_tensor)

    # 转换热力图
    heatmap_array = predicted_heatmap.squeeze().cpu().numpy()

    # 寻找最大值位置
    max_y, max_x = np.unravel_index(np.argmax(heatmap_array), heatmap_array.shape)

    # 坐标反缩放
    original_x = max_x * (orig_width / IMAGE_DIMENSION)
    original_y = max_y * (orig_height / IMAGE_DIMENSION)

    return original_x, original_y


def run_inference(trained_model):
    """运行测试集推理"""
    # 准备测试数据
    test_image_paths = []
    test_identifiers = []

    for img_num in range(81, 101):
        image_file = f"{img_num:04d}.jpg"
        full_test_path = os.path.join(testing_images_dir, image_file)

        if os.path.exists(full_test_path):
            test_image_paths.append(full_test_path)
            test_identifiers.append(img_num)

    # 预测每张图像
    prediction_results = {}
    for test_path, img_id_val in zip(test_image_paths, test_identifiers):
        pred_x, pred_y = estimate_coordinates(trained_model, test_path)
        prediction_results[img_id_val] = (pred_x, pred_y)
        print(f"图像 {img_id_val}: 预测X={pred_x:.2f}, 预测Y={pred_y:.2f}")

    # 生成提交文件
    submission_data = pd.read_csv(submission_template)

    for img_id_val, (pred_x, pred_y) in prediction_results.items():
        x_column = f"{img_id_val}_Fovea_X"
        y_column = f"{img_id_val}_Fovea_Y"

        submission_data.loc[submission_data['ImageID'] == x_column, 'value'] = pred_x
        submission_data.loc[submission_data['ImageID'] == y_column, 'value'] = pred_y

    # 保存结果
    submission_data.to_csv('/kaggle/working/final_predictions.csv', index=False)
    print("\n预测结果已保存至 final_predictions.csv")

    return submission_data


# -------------------------- 4. 程序主入口 --------------------------
def program_entry():
    print("开始执行中央凹定位任务...")

    # 训练模型
    print("\n阶段一: 模型训练")
    trained_network = execute_training()

    # 加载最佳模型
    print("\n阶段二: 加载最优模型")
    trained_network.load_state_dict(torch.load('optimal_model_weights.pth'))
    trained_network = trained_network.to(compute_device)

    # 运行推理
    print("\n阶段三: 测试集推理")
    final_predictions = run_inference(trained_network)

    # 显示部分结果
    print("\n预测结果预览:")
    print(final_predictions.head(10))



if __name__ == "__main__":
    program_entry()

运行在: cuda
开始执行中央凹定位任务...

阶段一: 模型训练


训练轮次 1/50: 100%|██████████| 8/8 [00:03<00:00,  2.04it/s, 批次MSE=0.024376, 批次RMSE=0.156128, 批次像素RMSE=454.86, 平均损失=0.068011]


✓ 保存最优模型，像素RMSE: 782.68，验证MSE: 0.00444054

====== 第 1 轮统计 ======
训练集 - 平均MSE: 0.06801052, 平均RMSE: 0.26078826, 平均像素RMSE: 552.55, 最小批次MSE: 0.02437582, 最大批次MSE: 0.15639549
验证集 - 平均MSE: 0.00444054, 平均RMSE: 0.06663735, 平均像素RMSE: 782.68, 最小批次MSE: 0.00442278, 最大批次MSE: 0.00445829



训练轮次 2/50: 100%|██████████| 8/8 [00:03<00:00,  2.05it/s, 批次MSE=0.009251, 批次RMSE=0.096181, 批次像素RMSE=960.30, 平均损失=0.014304] 


✓ 保存最优模型，像素RMSE: 554.63，验证MSE: 0.01026423

====== 第 2 轮统计 ======
训练集 - 平均MSE: 0.01430417, 平均RMSE: 0.11960004, 平均像素RMSE: 852.37, 最小批次MSE: 0.00925086, 最大批次MSE: 0.02083212
验证集 - 平均MSE: 0.01026423, 平均RMSE: 0.10131256, 平均像素RMSE: 554.63, 最小批次MSE: 0.01021058, 最大批次MSE: 0.01031789



...


====== 第 48 轮统计 ======
训练集 - 平均MSE: 0.00028252, 平均RMSE: 0.01680846, 平均像素RMSE: 10.14, 最小批次MSE: 0.00022818, 最大批次MSE: 0.00033573
验证集 - 平均MSE: 0.00050879, 平均RMSE: 0.02255629, 平均像素RMSE: 169.87, 最小批次MSE: 0.00042741, 最大批次MSE: 0.00059016



训练轮次 49/50: 100%|██████████| 8/8 [00:03<00:00,  2.05it/s, 批次MSE=0.000257, 批次RMSE=0.016047, 批次像素RMSE=7.95, 平均损失=0.000281] 



====== 第 49 轮统计 ======
训练集 - 平均MSE: 0.00028052, 平均RMSE: 0.01674869, 平均像素RMSE: 9.95, 最小批次MSE: 0.00022194, 最大批次MSE: 0.00033718
验证集 - 平均MSE: 0.00049745, 平均RMSE: 0.02230352, 平均像素RMSE: 169.67, 最小批次MSE: 0.00042505, 最大批次MSE: 0.00056984



训练轮次 50/50: 100%|██████████| 8/8 [00:03<00:00,  2.04it/s, 批次MSE=0.000261, 批次RMSE=0.016166, 批次像素RMSE=8.64, 平均损失=0.000270] 



====== 第 50 轮统计 ======
训练集 - 平均MSE: 0.00026985, 平均RMSE: 0.01642708, 平均像素RMSE: 9.11, 最小批次MSE: 0.00022065, 最大批次MSE: 0.00041522
验证集 - 平均MSE: 0.00049383, 平均RMSE: 0.02222229, 平均像素RMSE: 169.69, 最小批次MSE: 0.00040989, 最大批次MSE: 0.00057777


阶段二: 加载最优模型

阶段三: 测试集推理
图像 81: 预测X=1776.50, 预测Y=1156.25
图像 82: 预测X=1694.69, 预测Y=1078.12
图像 83: 预测X=1484.31, 预测Y=1039.06
图像 84: 预测X=1262.25, 预测Y=1101.56
图像 85: 预测X=1214.86, 预测Y=974.55
图像 86: 预测X=1273.94, 预测Y=1109.38
图像 87: 预测X=1484.31, 预测Y=1007.81
图像 88: 预测X=1285.62, 预测Y=1101.56
图像 89: 预测X=1484.31, 预测Y=1039.06
图像 90: 预测X=1647.94, 预测Y=1085.94
图像 91: 预测X=1484.31, 预测Y=1031.25
图像 92: 预测X=1706.38, 预测Y=1109.38
图像 93: 预测X=841.50, 预测Y=257.81
图像 94: 预测X=1484.31, 预测Y=1054.69
图像 95: 预测X=1694.69, 预测Y=1093.75
图像 96: 预测X=1729.75, 预测Y=1078.12
图像 97: 预测X=1507.69, 预测Y=1218.75
图像 98: 预测X=1472.62, 预测Y=1070.31
图像 99: 预测X=1484.31, 预测Y=1015.62
图像 100: 预测X=1706.38, 预测Y=1109.38

预测结果已保存至 final_predictions.csv

预测结果预览:
      ImageID        value
0  81_Fovea_X  1776.500000
1  81_Fovea